In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import bcolz
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
import math
import pickle
from tqdm import tqdm
from torch.autograd import Variable
# plotting
import matplotlib
matplotlib.use('Agg')
from vad import batchData
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

"""
Set seed #
"""
seed = 1337

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def loadDataset(path = '../Datasets/Reviews/dataset_ready.pkl'):
    return pickle.load(open(path, 'rb'))

In [2]:
print("Loading parameters..", end=" ")
hiddenSize = 512
latentSize = 400
batchSize  = 32
iterations = 3
learningRate = 0.0001
bidirectionalEncoder = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Done.")

print("Loading dataset..", end=" ")
dataset = loadDataset()
# setup store parameters
id2word = dataset['id2word']
word2id = dataset['word2id']
weightMatrix = dataset['weights']
train = dataset['train']
validation = dataset['validation']
cutoff = dataset['cutoff']
paddingID = word2id['<pad>']
print("Done.")

print("Converting dataset weights into tensors..", end=" ")
# convert dataset into tensors
weightMatrix = torch.tensor(weightMatrix, dtype=torch.float)
print("Done.")

# batching data
print("Batching Data..",end=" ")

# shuffle data rows
random.shuffle(train)
random.shuffle(validation)

trainx = [x[0] for x in train]
trainy = [x[1] for x in train]
valx = [x[0] for x in validation]
valy = [x[1] for x in validation]

# shuffle data row

trainx = batchData(trainx, paddingID, device, batchSize, cutoff)
trainy = batchData(trainy, paddingID, device, batchSize, cutoff)
valx = batchData(valx, paddingID, device, batchSize, cutoff)
valy = batchData(valy, paddingID, device, batchSize, cutoff)

traindata = (trainx, trainy)
valdata= (valx, valy)
print("Done.")

# setup variables for model components initialisation
maxReviewLength = cutoff
vocabularySize = len(id2word)
embeddingDim = weightMatrix.shape[1]
embedding_shape = weightMatrix.shape

Loading parameters.. Done.
Loading dataset.. Done.
Converting dataset weights into tensors.. Done.
Batching Data.. Done.


In [3]:
from vad import Encoder, Decoder, Attn, Backwards, Prior, Inference, loss_function

print("Initialising model components..", end=" ")

modelEncoder = Encoder(weightMatrix, vocabularySize,
                       paddingID, hiddenSize, bidirectionalEncoder).to(device)
# modelAttention = Attention(maxLength=maxReviewLength).to(device)
modelAttention = Attn(hiddenSize, bidirectionalEncoder).to(device)
modelBackwards = Backwards(weightMatrix, vocabularySize,
                           paddingID, hiddenSize, bidirectionalEncoder).to(device)
modelInference = Inference(
    hiddenSize, latentSize, bidirectionalEncoder).to(device)
modelPrior = Prior(hiddenSize, latentSize, bidirectionalEncoder).to(device)
modelDecoder = Decoder(weightMatrix, vocabularySize,
                       paddingID, batchSize, maxReviewLength, hiddenSize, latentSize, bidirectionalEncoder).to(device)
criterion = nn.NLLLoss(ignore_index=paddingID)
print("Done.")


# load models
print("Loading model weights..", end=" ")
modelEncoder.load_state_dict(torch.load('encoder.pth'))
modelAttention.load_state_dict(torch.load('attention.pth'))
modelBackwards.load_state_dict(torch.load('backwards.pth'))
modelInference.load_state_dict(torch.load('inference.pth'))
modelPrior.load_state_dict(torch.load('prior.pth'))
modelDecoder.load_state_dict(torch.load('decoder.pth'))
print("Done.")

Initialising model components.. Done.
Loading model weights.. Done.


In [4]:
def evaluate(x,
             xLength,
             y,
             yLength,
             encoder,
             attention,
             backwards,
             inference,
             prior,
             decoder,
             criterion
            ):
    
    loss = 0
    
    # initalise input and target lengths
    inputLength = x[0].size(0)
    targetLength = y[0].size(0)
    batchSize = x.shape[0]

    # set up encoder computation
    encoderHidden = encoder.initHidden(batchSize).to(device)
    backwardHidden = backwards.initHidden(batchSize).to(device)
    
    # set up encoder outputs
    encoderOutputs, encoderHidden = encoder(x, encoderHidden, xLength)

    # compute backwards outputs
    backwardOutput, backwardHidden = backwards(torch.flip(
        y, [0, 1]), yLength, backwardHidden)

    # set up the variables for decoder computation
    decoderInput = torch.tensor([[word2id["<sos>"]]] * batchSize, dtype=torch.long, device=device)
    
    decoderHidden = encoderHidden[-1]
    decoderOutput = None
    decoderOutputs = []
    
    # Run through the decoder one step at a time. This seems to be common practice across
    # all of the seq2seq based implementations I've come across on GitHub.
    for t in range(yLength[0]):
        # get the context vector c
        c = attention(encoderOutputs, decoderHidden)
        
        # compute the inference layer
        z_infer, infer_mu, infer_logvar = inference(decoderHidden, c, backwardOutput[:,t])
        
        # compute the prior layer
        z_prior, prior_mu, prior_logvar = prior(decoderHidden, c)
    
        # compute the output of each decoder state
        DecoderOut = decoder(decoderInput, c, z_prior, decoderHidden)
        
        # update variables
        decoderOutput, decoderHidden = DecoderOut
        decoderOutputs.append(decoderOutput)
        seqloss = loss_function(decoderOutput, y[:, t], infer_mu, infer_logvar, prior_mu, prior_logvar, criterion)
        loss += seqloss

        # feed this output to the next input
        decoderInput = y[:,t]
        decoderHidden = decoderHidden.squeeze(0)
        
    loss = loss.item()/targetLength
        
    return decoderOutputs, loss

In [28]:
def evaluateModel(batched_data,
                   encoder,
                   backward,
                   attention,
                   inference,
                   prior,
                   decoder,
                   criterion,
                   id2word,
                  ):
    
    encoder.eval()
    backward.eval()
    attention.eval()
    inference.eval()
    prior.eval()
    decoder.eval()
    numbatches = len(batched_data)
    for batch in tqdm(range(0,3)):
        # each batch is composed of the 
        # reviews, and a sentence length.
        x, xLength = batched_data[0][batch][0], batched_data[0][batch][1]
        y, yLength = batched_data[1][batch][0], batched_data[1][batch][1]
        
        outputs, losses = evaluate(x,
                                   xLength,
                                   y,
                                   yLength,
                                   encoder,
                                   attention,
                                   backward,
                                   inference,
                                   prior,
                                   decoder,
                                   criterion)
        
        print(y.shape)
#         reference = convertRealID2Word(id2word,y)
#         for e in reference:
#             print(e)
        words = convertDecoderID2Word(id2word,outputs)

        print("LOSS:", losses)
#         break

def convertRealID2Word(id2word, y):
    return [[id2word[x.item()] for x in y[entry].cpu()] for entry in range(len(y))]
        
def convertDecoderID2Word(id2word, outputs):
    entries = []
    for batch_line in outputs:
        entry = [torch.argmax(batch_line[i]).cpu().item() for i in range(len(batch_line))]
        entries.append([id2word[i] for i in entry])
        
    for i in range(len(outputs[0])):
        line = " ".join([entries[j][i] for j in range(12,len(entries))])
        line = line.replace(" ' ", "'")
        line = line.replace(" ) ", ")")
        line = line.replace(" . ", ".")
        line = line.replace(" , ", ", ")
        line = line.replace(" ( ", "(")
        print(line)
#         break
    return entries




In [29]:
outputs = evaluateModel(valdata,
                   modelEncoder,
                   modelBackwards,
                   modelAttention,
                   modelInference,
                   modelPrior,
                   modelDecoder,
                   criterion,
                   id2word
                  )

  0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([32, 60])


 33%|███▎      | 1/3 [00:01<00:02,  1.14s/it]

i'' t know to problem with the, ,, products, the, but'the)) but the <unk> <unk>, but, , <unk> the)).i i are be least of a lot more to the <unk>.but
i is the to for the the time <unk> <unk> of the )')))). the, <unk> of <unk> <unk> <unk> &.. the all, i's a great of to, is be be a.of.the
i'a a problems with, time, had it the, my, the <unk>, <unk>)<unk>)) <unk>, <unk>), <unk>'s a a <unk> <unk> for the <unk> <unk> <unk> <unk>, the <unk> <unk> <unk> <unk> <unk>
i'' a <unk> to the computer and, but i i computer phone is a longer, it the computer, a problem, i'' have to the else'to <eos> the <unk>, i'' use the the on the computer
i with the <unk> of to, and i'the to'find the to and)of)), you'' t have to to but i reviewers the other devices.. been.are the.they the.cards.<eos>
i are't a of, , i the <unk> of the <unk>, the other of, but'm say the <unk> wire, you'' t work.. of <eos>'' t had the <unk> in the.. be the
polarity_0.0 have have a $ <unk> replace 3 and(<unk> 3, the 3.0., to to the <unk>.

 67%|██████▋   | 2/3 [00:02<00:01,  1.14s/it]

if'this is a a'' t work a problem to the, use it to try, the i <unk> are is not much the.. i the of of.the <unk>.have't use to be the to.the
polarity_0.0 a'using with the product.the for to the.is the the months of use.<eos> have able the 7, a years the <unk> and and.. ago.i got to new of.. the.5..
i <unk> is is me issues, and the'not a than.the the <unk>.the.the was the <unk> of.the i was the <unk> is the <unk>, had the the i was the the the <unk> weeks.. i
the luck, i's a heavy to the way, not of, the are no way to be in <unk>.the little...the but the.and can to use the volume.the own.and the <unk>.the
polarity_0.0 been'm had it <unk> and and the computer, the the the to the, my'the the.computers.. the past...the'to use a few amount of time.get.<unk>.. the <unk>
i the, the i sure you you <unk> is is is can to to a to.. 0.. and can need the you.the reviews.the users.. you the can't be to problems.... <eos>
i's not good, but it'not a a of to the the the, i good good touch and, and, , t

100%|██████████| 3/3 [00:03<00:00,  1.13s/it]

i % the'the <unk> to the, , i comes to be the and i the <unk> <unk>, the, ,, , <unk>), <unk> the))).'to one a issue choice to get the <unk>.. .
i is the, and <unk> <unk>, <unk>, <unk>), is is a small easier than the',, is to't have you <unk> to use it of ,.. the than.. the and as a other, <unk>
i, <unk> is the.. <unk> is is a good, the, the'it'much more the.the.. the <unk> i is the the cover cover, , the.., is the.. of get out is
i have i is is a to be a <unk> of, but i extra of 20 for the of the one to the.i i i is is like a to you have the 3.0.the to <unk>.you't have
i are re not a, the same of of, , but with be the good to.the <unk> of to the.the <unk>...<unk>.the the.. though you <unk> is of)a the)<unk>))
i's a a a'it be able the the, you, to the <unk> inch is is be a, but it the you issue for ,.card.a.. are)))lot of than the <unk>)card
i the the the, i have the the hour <unk> <unk> <unk>'not research <unk> the <unk> com <unk> <unk> reviewers reviewers, the...<unk> the the the <unk